In [78]:
import plotly.offline as pox
import plotly.graph_objs as go
import numpy as np
import talib as tl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import talib as ta
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import alpaca_trade_api as tradeapi
from alpaca_trade_api import TimeFrame, TimeFrameUnit
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from matplotlib.pyplot import figure
from statsmodels.tsa.stattools import adfuller
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from math import exp

In [79]:
SEC_KEY = 'IBpWZnzOVLT8nUNZMwbfWg9oV0sZrkAHPQYtsRFP'
PUB_KEY ='PKBEIUYRXMQIO5JXOBAJ'
BASE_URL ='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL) # For real trading, don't enter a base_url
df = api.get_bars("AAPL", TimeFrame(1, TimeFrameUnit.Day), "2016-01-01", "2023-01-01").df # Get one bar object for each of the past 5 minutes
df.reset_index(inplace=True)
df.head()

,timestamp,open,high,low,close,volume,trade_count,vwap
0,2016-01-04 05:00:00+00:00,102.61,105.368,102.00,105.35,67611658,351450,104.048524
1,2016-01-05 05:00:00+00:00,105.75,105.850,102.41,102.71,55787192,321363,103.415766
2,2016-01-06 05:00:00+00:00,100.48,102.370,99.87,100.70,68340388,409162,100.852488
3,2016-01-07 05:00:00+00:00,98.63,100.130,96.43,96.45,81094357,462834,98.069324
4,2016-01-08 05:00:00+00:00,98.55,99.110,96.76,96.96,70797966,419804,97.923007


In [80]:
df["close"] =  df['close'] * 100
df["high"]  =  df['high']  * 100
df["low"]   =  df['low']   * 100
split_percentage = 0.8
split = int(split_percentage*len(df))
a =  df[split:]
a.head()

,timestamp,open,high,low,close,volume,trade_count,vwap
1409,2021-08-09 04:00:00+00:00,146.20,14670.00,14552.0,14609.0,48857702,351878,146.111166
1410,2021-08-10 04:00:00+00:00,146.44,14771.00,14530.0,14560.0,68980001,430863,146.126086
1411,2021-08-11 04:00:00+00:00,146.05,14672.00,14553.0,14586.0,48420536,364779,146.022670
1412,2021-08-12 04:00:00+00:00,146.19,14905.00,14584.0,14889.0,73757305,514473,148.010325
1413,2021-08-13 04:00:00+00:00,148.97,14944.44,14827.0,14910.0,59352311,407701,148.919361


In [81]:
df['future_returns'] = df['close'].pct_change().shift(-1)
signal=[]
for i in range(len(df)):
            if (df['future_returns'][i]  > 0):
                signal.append(1)
            elif (df['future_returns'][i] < 0):
                signal.append(0)
            else:
                signal.append(0)
df['signal']=signal

df['High - Close']=(df['high']- df["close"]).shift(-1)
df['Close - Low']=(df["close"]- df['low']).shift(-1)
df['df3'] = df['High - Close'] * df['Close - Low']
df.dropna(inplace=True)
y = df[['signal']].copy()
X = df[['Close - Low','High - Close']].copy()

In [82]:
import time
split_percentage = 0.8
split = int(split_percentage*len(df))
# Train data set
X_train = X[:split]
y_train = y[:split]
X_test =  X[split:]
y_test =  y[split:]
model = KNeighborsClassifier(n_neighbors= 9)
model.fit(X_train, y_train['signal'])
start_time = time.time()

y_pred = model.predict(X_test)
# get end time
end_time = time.time()

# calculate elapsed time
elapsed_time = end_time - start_time
Accuracy =accuracy_score(y_test, y_pred)
print(Accuracy)
elapsed_time

0.8356940509915014


0.028025388717651367

In [83]:
X_train.to_csv("X_train.csv")
X_test.to_csv("X_test.csv")
y_test.to_csv("y_test.csv")

In [84]:
y_pred

array([0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1,

# Basic KNN

In [85]:
Temp = pd.DataFrame() 
Temp1 = pd.DataFrame() 
Temp['X_test'] = X_test
Temp1['X_train'] = X_train
# for i in range(len(X_test)):
for j in range(len(X_train)):
    Temp1ist'][j] = ['dTemp['X_test'].iloc[1] - Temp1['X_train'].iloc[j]

Temp1['dist'].tail()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3367581813.py, line 7)